# Objective Evaluation

## Definition of the Shape Optimization Objective

## Effective Evaluation of the AOT'd Simulation

## Design Optimization

### Gradient-based Optimization

### GFlow-Nets

In [ ]:
# Dump from the gradient notebook
input_reader = InputReader(case_dict, numerical_setup_dict)
initializer  = Initializer(input_reader)
sim_manager  = SimulationManager(input_reader)

# PRE SHOCK CONDITIONS
gamma_L, gamma_R = 1.4
rho_R = p_R = 1.0
a_R   = np.sqrt(gamma_R * p_R / rho_R)
u_R   = 0.0
M_R   = u_R / a_R

@jax.jit
def fun(M_S: float = 2.0):
    traj_length = 5
    time_step   = 1e-2
    res = case_dict["nx"]

    dx = 1.0 / res
    x_cf   = jnp.linspace(0, 1, num=res+1)
    x_cc = 0.5 * (x_cf[1:] + x_cf[:-1])

    # POST SHOCK RANKINE HUGONIOT CONDITIONS
    p_L   = p_R * ( 1/(gamma_L + 1) * (gamma_R * (M_R - M_S)**2 + 1) + jnp.sqrt( (1/(gamma_L + 1) * (gamma_R * (M_R - M_S)**2 + 1))**2 - (gamma_L-1)/(gamma_L+1) * ((M_R-M_S)**2 * 2 * gamma_R/(gamma_R - 1) - 1) ))
    rho_L = rho_R *  (gamma_R - 1)/(gamma_L - 1) * ( p_L / p_R + (gamma_L - 1)/ (gamma_L + 1) ) / ( p_L / p_R * (gamma_R - 1) / (gamma_L + 1) + (gamma_R + 1) / (gamma_L + 1) )
    u_L   = a_R * ( rho_R/rho_L * (M_R - M_S) + M_S )

    # INTIAL BUFFER
    prime_init      = jnp.zeros((1, 5, res, 1, 1))
    prime_init      = prime_init.at[0,0,:,0,0].set(jnp.where(x_cc > 0.5, rho_R, rho_L))
    prime_init      = prime_init.at[0,1,:,0,0].set(jnp.where(x_cc > 0.5, u_R, u_L))
    prime_init      = prime_init.at[0,4,:,0,0].set(jnp.where(x_cc > 0.5, p_R, p_L))
    levelset_init   = None

    # FORWARD SIMULATION
    data_series, _ = sim_manager.feed_forward(
        prime_init,
        levelset_init,
        traj_length,
        time_step,
        0.0, 1, None, None)
    data_series = data_series[0]

    # COMPUTE SCALAR OUTPUT QUANTITY
    entropy = data_series[:,4] / data_series[:,0]**gamma_L
    total_entropy = jnp.mean(data_series[-1,0] * entropy[-1] - data_series[0,0] * entropy[0])
    return total_entropy